In [1]:
!pip install unionllm
!pip install zeval

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
# install the required packages
from unionllm import unionchat
from datasets import Dataset
from zeval.evaluation import evaluate
from zeval.evaluators import Faithfulness, AnswerRelevancy, ContextRecall, ContextPrecision
# from evalsone import EvalsOne

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# query Coze with a query and get the completion and the context
query = "智商测试"
ideal = ["瑞文标准推理测验（SPM）是一种广受认可的智力测验工具，由英国心理学家瑞文于1938年创制。该测验是一种纯粹的非文字智力测验，广泛应用于无国界的智力/推理能力测试。它基于斯皮尔曼的智力二因素论，认为智力主要由普通因素（G因素）和特殊因素（S因素）构成。瑞文测验主要测量“G”因素，与问题解决、清晰知觉和思维、发现和利用信息以及适应社会生活能力相关。该测验由60张图组成，分为5个单元，考察了知觉辨别、类同比较、比较推理、系列关系推理、抽象推理运算等方面的思维能力。"]
messages = [
    {"role": "user", "content": "智商测试"}
]

params = {
    "provider": "coze",
    "api_key": "pat_hodyfcRog0CfCAfEMtKAueI1I8yD2tDuDBSrvSRoXhKm1xLCw0HDiePtEVgRFHzt",
    "stream": False,
    "details": True
}

model = "7355829428481769477"
response = unionchat(model=model, messages=messages, **params)
print(response)

history is: []
payload is: {"query": "\u667a\u5546\u6d4b\u8bd5", "user": "6181924e67d857563", "bot_id": "7355829428481769477", "chat_history": [], "steam": false}
messages is: [{'role': 'assistant', 'type': 'verbose', 'content': '{"verbose_type":"knowledge","prompt":"\\n# 回答规则\\n根据引用的内容回答问题:\\n1.如果引用的内容里面包含 \\u003cimg src=\\"\\"\\u003e 的标签, 标签里的 src 字段表示图片地址, 可以在回答问题的时候展示出去, 输出格式为\\"![图片名称](图片地址)\\" \\n2.如果引用的内容不包含 \\u003cimg src=\\"\\"\\u003e 的标签, 你回答问题时不需要展示图片 \\n\\n# 引用的内容 \\nsection 1 start:\\nhigh priority:\\n sql: SELECT * FROM processed_questions_and_answers WHERE question LIKE \'%智商测试%\' OR answer LIKE \'%智商测试%\'\\n result: [{\\"answer\\":\\"瑞文智力测验的题目对于高智力人群的区分度有限，因此当受测者的智力高于135时，我们统称之为135+。建议您寻找其他测试量表获得更具体的智商得分，例如韦氏智力测试等。我们也提供一套瑞文高级智力测验，可供您参考选择。但受限于试题本身的知识维度、测验方式、测试环境、试题数量、及出题时间等多种因素，任何智力测验都有一定的局限性，难以做到绝对精确的测量，因此结果仅供参考。\\",\\"question\\":\\"智商测试报告显示是135+以上，能获得具体的数值吗？\\"}]\\nsection 1 ends.\\nsection 2 start:\\n{\\"answer\\":\\"心理测评为虚拟产品，购买后一般不支持退款，请您理解。如果您有特殊的理由，请从微信支付里找到交易的订单

In [4]:
# create a dataset from the response for evaluation
sampled = response.choices[0].message.content
context = []
for context_item in response.context:
    context.append(context_item.content)

dataset = Dataset.from_dict({
    'question': [query],
    'answer': [sampled], 
    'context': [context],
    'ideal': [ideal]
})
print(dataset['context'])

[['high priority:\n sql: SELECT * FROM processed_questions_and_answers WHERE question LIKE \'%智商测试%\' OR answer LIKE \'%智商测试%\'\n result: [{"answer":"瑞文智力测验的题目对于高智力人群的区分度有限，因此当受测者的智力高于135时，我们统称之为135+。建议您寻找其他测试量表获得更具体的智商得分，例如韦氏智力测试等。我们也提供一套瑞文高级智力测验，可供您参考选择。但受限于试题本身的知识维度、测验方式、测试环境、试题数量、及出题时间等多种因素，任何智力测验都有一定的局限性，难以做到绝对精确的测量，因此结果仅供参考。","question":"智商测试报告显示是135+以上，能获得具体的数值吗？"}]', '{"answer":"心理测评为虚拟产品，购买后一般不支持退款，请您理解。如果您有特殊的理由，请从微信支付里找到交易的订单号(420000开头的一串数字)发送给我们并说明申请退款的理由，我们收到并核实后会与您联系。","question":"智力测验退款"}', '{"answer":"","question":"瑞文标准智力测验：你有多聪明？,http://p.melbacastillo.top/pquiz/rend/2725039?retake=1\\u0026cstr=5c987b7c4001f"}', '{"answer":"瑞文标准智力测验是国际上广受认可的智力测验工具。它是一种纯粹的非文字智力测验，所以广泛应用于无国界的智力/推理能力测试。具有较高的信度和效度。《最强大脑》中采用的就是这种智力测量标准。瑞文智力测验结果与测试参与者在测验时的精神状态，周边环境等多种因素有关，波动范围大约在3至15分左右（百分比等级分数）。","question":"瑞文智力测验的结果可信吗？"}']]


In [5]:
# specify the evaluation parameters
eval_params = {
    "provider": "zhipuai",
    "model": "glm-4",
    "api_key": "38dc046c622b6ec1b9bfa0413e6ca2ee.2Yn3uFr8j74pMOvK",
    "temperature": 0.1,
    "stream": False
}

In [6]:
# evaluate the dataset with the specified evaluator and print the results
faithfulness_results = evaluate(dataset, [Faithfulness], eval_params)
print(faithfulness_results)
print(faithfulness_results.reasoning)

INFO:root:messages: [{'content': "    给定一个问题和答案，请从给定答案的每个句子中创建一个或多个陈述。\n    问题：阿尔伯特·爱因斯坦是谁，他最著名的是什么？\n    答案：他是一名出生在德国的理论物理学家，被广泛认为是有史以来最伟大和最有影响力的物理学家之一。他最著名的是发展相对论理论，他还为量子力学理论的发展做出了重要贡献。\n    陈述：\n    阿尔伯特·爱因斯坦出生在德国。\n    阿尔伯特·爱因斯坦以他的相对论理论而闻名。\n    问题：氯化镉在这种化学物质中略微可溶，它还被称为什么？\n    答案：酒精\n    陈述：\n    氯化镉在酒精中略微可溶。\n    问题：Shahul和Jithin是同一国籍吗？\n    答案：他们来自不同的国家。\n    陈述：\n    Shahul和Jithin来自不同的国家。\n    问题：['智商测试']\n    答案：['智商测试是衡量一个人智力水平的工具。我可以提供一些关于智商测试的信息。\\n\\n一款常用的智力测验是瑞文标准智力测验，这是一种被广泛认可的智力测验工具。瑞文标准智力测验是一种纯粹的非文字智力测验，对无国界的智力/推理能力进行测试，具有较高的信度和效度。在《最强大脑》中也采用了这种智力测量标准。瑞文智力测验的结果与被测者在测验时的精神状态，周边环境等多种因素有关，波动范围大约在3至15分左右（百分比等级分数）。\\n\\n如果你对瑞文标准智力测验感兴趣，你可以从这个链接进行测验：[你有多聪明？](http://p.melbacastillo.top/pquiz/rend/2725039?retake=1\\\\u0026cstr=5c987b7c4001f)。\\n\\n需要注意，智商测试报告如果显示是135+以上，那么这个测试的区分度有限，因此当受测者的智力高于135时，我们统称之为135+。如果你希望获得更具体的智商得分，建议使用其他测试量表，例如韦氏智力测试等。我们也提供一套瑞文高级智力测验，可供你参考选择。但请记住，任何智力测验都有一定的局限性，因此结果应仅供参考。\\n\\n此外，如果你已经购买了心理测评，需要注意这类虚拟产品一般是不支持退款的。如果有特殊理由需要退款，可以找到交易的订单号(420000开头的一串数字)

['是', '是', '是', '是', '是', '是', '是']
raw score:  0.0
Result(score=1.0, reasoning='\n    我的推理过程如下：\n    首先，我从给定答案的每个句子中创建以下的陈述：\n    智商测试是用于评估个人智力水平的工具。\n瑞文标准智力测验是一种广泛使用的非文字智力测验，它测试的是无国界的智力/推理能力，并且具有较高的信度和效度。\n瑞文标准智力测验的结果可能会受到被测者精神状态和周边环境等因素的影响，波动范围大约在3至15分左右。\n智商测试中，如果分数超过135，通常会被标记为135+，表明测试在高智商区间的区分度有限。\n为了获得更具体的智商得分，建议使用如韦氏智力测试等其他测试量表。\n任何智力测验都有局限性，因此测试结果应仅作为参考。\n如果已经购买了心理测评服务，需要注意这类虚拟产品通常不支持退款，但在有特殊理由时可以申请退款。\n    然后，我考虑了上下文信息，并对每个陈述进行了自然语言推理。最后，我得出了每个陈述的最终裁决。\n    1.智商测试是用于评估个人智力水平的工具。\n    解释：上下文中提到智商测试，并解释了它是如何用来评估个人智力水平的。裁决：是。\n\n    2.瑞文标准智力测验是一种广泛使用的非文字智力测验，它测试的是无国界的智力/推理能力，并且具有较高的信度和效度。\n    解释：上下文明确提到瑞文标准智力测验是国际上广受认可的智力测验工具，具有高信度和效度，用于无国界的智力/推理能力测试。裁决：是。\n\n    3.瑞文标准智力测验的结果可能会受到被测者精神状态和周边环境等因素的影响，波动范围大约在3至15分左右。\n    解释：上下文提到瑞文智力测验的结果会受到多种因素的影响，包括测试参与者的精神状态和周边环境，且有波动范围。裁决：是。\n\n    4.智商测试中，如果分数超过135，通常会被标记为135+，表明测试在高智商区间的区分度有限。\n    解释：上下文提到当智力测试分数高于135时，会标记为135+，并指出对于高智力人群的区分度有限。裁决：是。\n\n    5.为了获得更具体的智商得分，建议使用如韦氏智力测试等其他测试量表。\n    解释：上下文提供了这样的建议，即当瑞文智力测验无法提供具体分数时，可以考虑使用其他

In [ ]:
eo_client = EvalsOne(api_key='your_api_key_here')
kwargs = {'messages': 'example message', 'completion': 'example completion'}
response = eo_client.add_sample(**kwargs)
print(response)